In [1]:
import torch
import sys
sys.path.append('../')
from scipy.signal import medfilt
import numpy as np
import model as UNet
import torch.nn as nn
import matplotlib.pyplot as plt

In [ ]:
model = UNet.UNet1Sigmoid(1,1,32,25)
print(model)

In [ ]:
npz = np.load('data.npz')

from scipy.ndimage import uniform_filter

pre_flux = npz['flux']
filtered_flux = uniform_filter(pre_flux, size=3, mode='reflect',axes=1)

max_vals = np.max(filtered_flux, axis=1)
flux = np.array([pre_flux[i,:] /max_vals[i] for i in range(0,len(max_vals))])

red_line = npz['gauss']+ npz['pl'] + npz['Fe']
mask = np.array([red_line[i,:] /max_vals[i] for i in range(0,len(max_vals))])


torch_array = torch.unsqueeze(torch.asarray(flux,dtype=torch.float32),dim=1)
torch_array.cuda()

In [ ]:
# 加载模型
model = UNet.UNet1Sigmoid(1,1,32,25)
model = nn.DataParallel(model, device_ids=[1])
model.load_state_dict(torch.load("/home/yaomingliang/work/uunet/2024_final/20240106001.pth"))
# model.set_to_eval()
# 调用模型
out = model(torch_array)
out_ = out.cpu()